In [1]:
from gensim.models import FastText
import gensim
import pandas as pd

DATA_PATH = '/data/'
MODEL_PATH = DATA_PATH + 'models/ft4_ru/'
SOURCE_PATH = DATA_PATH + 'captions/lem/'

cities = ['moscow', 'spb']#, 'nyc', 'london']
years = ['2016', '2017', '2018', '2019', '2020']
files = []
for city in cities:
    for year in years:
        files.append([city, year])

def csv_path(path, city, year):
    return path + city + '_posts_' + year + '.csv'

valid_langs = set(['__label__ru'])

USE_PRETRAINED_MODEL = False

In [2]:
!mkdir {MODEL_PATH}
class MyCorpus(object):
    iter = 0
    def __iter__(self):
        self.iter += 1
        for city in cities:
            for year in years:
                df = pd.read_csv(csv_path(SOURCE_PATH, city, year))
                df = df[df.lang.isin(valid_langs)]
                for s in df['caption']:
                    yield s.split()
                              
            print(f'iter {self.iter} for {city} completed')
                
        print(f'\niter {self.iter} completed\n')

sentences = MyCorpus()

# dimension 300, with character n-grams of length 5, a window of size 5 and 10 negatives
# https://fasttext.cc/docs/en/crawl-vectors.html

if USE_PRETRAINED_MODEL:
    model =  gensim.models.fasttext.load_facebook_model(MODEL_PATH + 'wiki.ru.bin')
else:
    model = FastText(size=300, window=15, min_count=10, word_ngrams=2, negative=20, workers=35)
print(f'success, vocab: {len(model.wv.vocab)}')

success, vocab: 0


In [ ]:
print('building vocab\n')
if USE_PRETRAINED_MODEL:
    model.build_vocab(sentences=MyCorpus(), update=True)
else:
    model.build_vocab(sentences=MyCorpus())
total_examples = model.corpus_count
print(f'\ncorpus count: {total_examples}; vocab: {len(model.wv.vocab)}\nend of building')

building vocab



In [ ]:
print(len(model.wv.vocab))

In [ ]:
print('start training\n')
model.train(sentences=MyCorpus(), total_examples=total_examples, epochs=40)
print(f'\nend of training')

In [ ]:
model.save(MODEL_PATH + 'mdl')

f = open(MODEL_PATH + 'description.txt', 'w')
f.write('date: 08.05\n')
f.write('params: size=300, min_count=1, window=5, negative=10, word_ngrams=5, epochs=5\n')
f.write(f'sourse: {SOURCE_PATH}\n')
f.write('cities: moscow, spb; only russian\n')
print('success')
f.close()

In [7]:
with open('tests.txt') as f:
    tests = [[word for word in line.split()] for line in f]

for test in tests:
    print(model.wv.similarity(test[0], test[1]), test[0], test[1])
    

0.80805814 8марта 23февраля
0.27433598 8марта выставка
0.48473305 8марта рождество
0.37270707 8марта свадьба
0.53255945 8марта 9мая
0.24352027 8марта сплин
0.49690336 сплин слот
0.57352257 сплин rammstein
0.32789782 сплин рождество
0.2025525 сплин свадьба
0.20832917 сплин маникюр
0.26205045 сплин спб
0.65433633 сплин ддт
0.60260904 сплин концерт
0.5566946 сплин песня
0.5037586 сплин нойз
0.6719568 зенит ска
0.61660373 зенит хоккей
0.6510804 зенит баскетбол
0.39403272 зенит сплин
0.28050265 зенит 8марта
0.30077258 зенит рождество
0.7404321 зенит футбол
0.6728875 зенит арена
0.48191747 зенит мяч
0.6609002 зенит гол
0.79878855 зенит спартак
0.19693191 зенит маникюр
0.16906635 зенит сушь
0.37501115 зенит тенис
0.46583143 тенис волебол
0.22568047 тенис марафон
0.23857808 тенис лыжа
0.38073522 зенит спорт
0.50807 тенис спорт
0.5412748 баскетбол спорт
0.6130132 итмо университет


In [7]:
with open('tests.txt') as f:
    tests = [[word for word in line.split()] for line in f]

for test in tests:
    print(model.wv.similarity(test[0], test[1]), test[0], test[1])

0.77465004 8марта 23февраля
0.26400387 8марта выставка
0.44828498 8марта рождество
0.3088644 8марта свадьба
0.47429362 8марта 9мая
0.19591044 8марта сплин
0.3752808 сплин слот
0.5153809 сплин rammstein
0.2440366 сплин рождество
0.17200556 сплин свадьба
0.12906846 сплин маникюр
0.14062892 сплин спб
0.6133099 сплин ддт
0.55933976 сплин концерт
0.56913275 сплин песня
0.50104636 сплин нойз
0.6024536 зенит ска
0.5754436 зенит хоккей
0.5914444 зенит баскетбол
0.32471004 зенит сплин
0.22286703 зенит 8марта
0.24321106 зенит рождество
0.7139603 зенит футбол
0.6243685 зенит арена
0.42467847 зенит мяч
0.6441635 зенит гол
0.7541795 зенит спартак
0.13084683 зенит маникюр
0.11200154 зенит сушь
0.36260748 зенит тенис
0.43142712 тенис волебол
0.16143346 тенис марафон
0.4329789 тенис лыжа
0.29186195 зенит спорт
0.43753514 тенис спорт
0.5064553 баскетбол спорт
0.6249611 итмо университет
